In [1]:
import Queue
import random
from random import shuffle, randint
from threading import Thread
import time
import numpy as np
import tensorflow as tf
import glob
import cPickle as pkl
from collections import namedtuple

In [2]:
import tensorflow as tf

In [3]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string("model", "", "The name of model runned.")
tf.app.flags.DEFINE_string("data_path", "", "Path expression to data file.")
tf.app.flags.DEFINE_string("input_vocab", "", "Path to input vocabulary file.")
tf.app.flags.DEFINE_string("output_vocab", "", "Path to output vocabulary file.")
tf.app.flags.DEFINE_integer("input_vsize", 0,"Number of words in input vocabulary.")
tf.app.flags.DEFINE_integer("output_vsize", 0,"Number of words in output vocabulary.")
tf.app.flags.DEFINE_string("ckpt_root", "", "Directory for checkpoint root.")
tf.app.flags.DEFINE_string("summary_dir", "", "Directory for summary files.")
tf.app.flags.DEFINE_string("mode", "train", "train/decode mode")
tf.app.flags.DEFINE_integer("batch_size", 1, "Size of minibatch. Beam size in decode mode.")
# ----------- Train mode related flags ------------------
tf.app.flags.DEFINE_float("lr", 0.15, "Initial learning rate.")
tf.app.flags.DEFINE_float("min_lr", 0.01, "Minimum learning rate.")
tf.app.flags.DEFINE_float("max_grad_norm", 1.0, "Maximum gradient norm for gradient clipping.")
tf.app.flags.DEFINE_integer("decay_step", 30000, "Exponential decay step.")
tf.app.flags.DEFINE_float("decay_rate", 0.1, "Exponential decay rate.")
tf.app.flags.DEFINE_integer("max_run_steps", 1000000, "Maximum number of run steps.")
tf.app.flags.DEFINE_float("dropout", 0.0, "Dropout rate.")
tf.app.flags.DEFINE_string("valid_path", "", "Path expression to validation set.")
tf.app.flags.DEFINE_integer("valid_freq", 1000, "How often to run eval.")
tf.app.flags.DEFINE_integer("num_valid_batch", 30, "Number valid batches in each _Valid step.")
tf.app.flags.DEFINE_integer("checkpoint_secs", 1200, "How often to checkpoint.")
tf.app.flags.DEFINE_integer("max_to_keep", None,
                            "Maximum number of checkpoints to keep. "
                            "Keep all by default")
tf.app.flags.DEFINE_integer("display_freq", 200, "How often to print.")
tf.app.flags.DEFINE_integer("verbosity", 20,
                            "tf.logging verbosity (default INFO).")
# ----------- Data reading related flags ------------------
tf.app.flags.DEFINE_bool("use_bucketing", False,
                         "Whether bucket articles of similar length.")
tf.app.flags.DEFINE_bool("truncate_input", False,
                         "Truncate inputs that are too long. If False, "
                         "examples that are too long are discarded.")
# ----------- Decode mode related flags ------------------
tf.app.flags.DEFINE_string("decode_dir", "", "Directory for decode summaries.")
tf.app.flags.DEFINE_integer("extract_topk", 3,
                            "Number of sentence extracted in decode mode.")
# ----------- general flags ----------------
tf.app.flags.DEFINE_integer("emb_dim", 128, "Dim of word embedding.")
tf.app.flags.DEFINE_integer("num_gpus", 1, "Number of gpus used.")
# ----------- summarunner related flags ----------------
tf.app.flags.DEFINE_integer("enc_layers", 1, "Number of encoder layers.")
tf.app.flags.DEFINE_integer("num_sents_doc", 100,
                            "Maximum number of sentences in a document.")
tf.app.flags.DEFINE_integer("num_words_sent", 50,
                            "Maximum number of words in a sentence.")
tf.app.flags.DEFINE_integer("rel_pos_max_idx", 11,
                            "Maximum index of relative position embedding.")
tf.app.flags.DEFINE_integer("enc_num_hidden", 512,
                            "Number of hidden units in encoder RNN.")
tf.app.flags.DEFINE_integer("pos_emb_dim", 64,
                            "Dimension of positional embedding.")
tf.app.flags.DEFINE_integer("doc_repr_dim", 512,
                            "Dimension of document representation.")
tf.app.flags.DEFINE_string("word_conv_k_sizes", "3,5,7",
                           "Kernel sizes of word-level CNN.")
tf.app.flags.DEFINE_integer("word_conv_filter", 128,
                            "Number of output filters of all kernel sizes.")
tf.app.flags.DEFINE_integer("min_num_input_sents", 0,
                            "Minimum number of sentences in input docuement.")
tf.app.flags.DEFINE_integer("min_num_words_sent", 0,
                            "Ignore sentences shorter than this threshold.")
tf.app.flags.DEFINE_integer("trg_weight_norm", 0,
                            "Normalize the extraction target weights. "
                            "No normalization if it is not positive.")
# ----------- summarunner_rf related flags ----------------
tf.app.flags.DEFINE_string("train_mode", "sl",
                           "Kernel sizes of word-level CNN.")
tf.app.flags.DEFINE_string("mlp_num_hiddens", "256", "Kernel sizes of word-level CNN.")
tf.app.flags.DEFINE_float("rl_coef", 1.0, "Coefficient for RL loss in SL+RL mode.")  # Hệ số mất mát


In [4]:
class Vocab:
    def __init__(self, word2id):
        self.word2id = word2id
        self.id2word = {v: k for k, v in word2id.items()}
        assert len(self.word2id) == len(self.id2word)
        self.PAD_IDX = 0
        self.UNK_IDX = 1
        self.PAD_TOKEN = 'PAD_TOKEN'
        self.UNK_TOKEN = 'UNK_TOKEN'

    def __len__(self):
        return len(self.word2id)

    def i2w(self, idx):
        return self.id2word[idx]

    def w2i(self, w):
        if w in self.word2id:
            return self.word2id[w]
        else:
            return self.UNK_IDX

    def get_ids(self, text):
        """Get ids corresponding to words in text.
    Assumes tokens separated by space.

    Args:
      text: a string with tokens separated by space.

    Returns:
      A list of ints representing word ids.
    """
        return [self.w2i(w) for w in text.split()]

In [5]:
import json

In [6]:
with open("data/word2id.json") as f:
    word2id = json.load(f)
    print(json.load(f))

In [7]:
_input= Vocab(word2id)

In [8]:
_input.__len__()

In [9]:
print(_input.__len__())